## 参考
- https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html
- https://towardsdatascience.com/conditional-random-field-tutorial-in-pytorch-ca0d04499463
- forward-backward algorithm: https://en.wikipedia.org/wiki/Forward%E2%80%93backward_algorithm
- Viterbi algorithm: https://en.wikipedia.org/wiki/Viterbi_algorithm
- http://blog.echen.me/2012/01/03/introduction-to-conditional-random-fields/
- http://www.cs.columbia.edu/~mcollins/crf.pdf

In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [4]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

ValueError: only one element tensors can be converted to Python scalars